In [ ]:
import glob
from sys import exit, path
from os.path import join, expanduser, exists

import numpy as np
import pandas as pd
import scipy.interpolate as spi
import scipy.signal as sps

from bokeh.io import output_file, export_png, export_svgs, show, output_notebook
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.models import ColorBar, ColumnDataSource, Span
from bokeh.layouts import gridplot
import bokeh.palettes
import colorcet as cc

import plotly.graph_objects as go
import plotly.offline as poff

path.insert(1, expanduser('~/src/noexiit/software/analyses'))
path.insert(1, expanduser('~/src/cmocean-bokeh'))

from cmocean_cmaps import get_all_cmocean_colours

output_notebook()

In [ ]:
cm = get_all_cmocean_colours()

In [ ]:
from analyze_fictrac import parse_dats
from analyze_fictrac import unconcat_df

# Parse and unconcatenate: 
pson_open = parse_dats("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 
                        1, 5, acq_mode="offline", do_confirm=False)
dcor_open = parse_dats("/mnt/2TB/data_in/HK_20200316/dcor_open_loop/", 
                        1, 5, acq_mode="offline", do_confirm=False)
pson_open_by_animal = unconcat_df(pson_open)
dcor_open_by_animal = unconcat_df(dcor_open)

# Stimulus:
from analyze_stimulus import parse_2dof_stimulus, merge_stimulus_with_data, make_stimulus_trajectory, plot_fictrac_XY_with_stim

# Use Platyusa open loop data as test:
stims = parse_2dof_stimulus("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 1, 0, 27, 27)
merged = merge_stimulus_with_data(stims, pson_open, fill_method="linear")
stimmed = make_stimulus_trajectory(merged)
pson_open_by_animal = unconcat_df(stimmed)

# Explore plots

This notebook is meant as an exploratory playground for generating plots. Its primary usefulness is its documentation of code snippets that generate plotting features, even though the overall plots themselves are usually not useful.

In [ ]:
from analyze_fictrac import plot_fictrac_XY_cmap

df_plot = pson_open_by_animal[7].dropna()
# df_plot = df_plot[::5]

p = plot_fictrac_XY_cmap(df_plot, 
                     cmap_col="dist_from_stim_mm", 
                     cmap_label="dist from stim (mm)", 
                     palette=bokeh.palettes.Blues256[150:20:-1], 
                     high_percentile=97, 
                     alpha=0.04, 
                     size=4, show_plots=False) 
# TODO: fix weird bug that makes a list
p[1].border_fill_color = "#f8f5f2"
p[1].xgrid.grid_line_color = "#efe8e2"
p[1].ygrid.grid_line_color = "#efe8e2"
p[1].title.text = "A Platyusa open loop with L. occidentale"

show(p[1])

In [ ]:
# Holoviews hovertool example with Bokeh:

import holoviews as hv
import bebi103
hv.extension('bokeh')
bebi103.hv.set_defaults()

hv.Points(data=df_plot, # consider displaying step-slice for memory efficiency
          kdims=["X_mm", "Y_mm"],
          vdims=['secs_elapsed', "integrat_animal_heading"],
         ).opts(tools=['hover'], width=800, height=800, size=3, alpha=0.1)

In [ ]:
def plot_fictrac_XY_cmap_with_stim(dfs, 
                                   low=0, 
                                   high_percentile=95, 
                                   respective=False, 
                                   
                                   cmap_col_beetle="speed_mm_s",
                                   cmap_label_beetle="mm/s (spd)", 
                                   palette_beetle=cm["thermal"], 
                                   
                                   cmap_col_stim="dist_from_stim_mm",
                                   cmap_label_stim="mm (stim dist)",
                                   palette_stim=bokeh.palettes.Greys256[150:30:-1], 
                                   
                                   size=2.0, 
                                   alpha=0.3, 
                                   show_start=False, 
                                   save_path=None, 
                                   show_plots=True):
    
    assert (low >= 0), \
        f"The low end of the colour map range must be non-negative"
    assert ("X_mm" in dfs), \
        f"The column, 'X_mm', is not in the input dataframe, {dfs}"
    assert ("Y_mm" in dfs), \
        f"The column, 'Y_mm', is not in the input dataframe, {dfs}"
    assert (cmap_col_beetle in dfs), \
        f"The column, {cmap_col_beetle}, is not in the input dataframe, {dfs}"
    assert ("animal" in dfs), \
            f"The column 'animal' is not in in the input dataframe, {dfs}"
    # TODO: Do same assertions for cmap_col_stim
    
    dfs_list = unconcat_df(dfs, col_name="animal")
    
    if respective is False:
        high_beetle = np.percentile(dfs[cmap_col_beetle], high_percentile)
        high_stim = np.percentile(dfs[cmap_col_stim], high_percentile)

    bokeh_ps = []
    for _, df in enumerate(dfs_list):
        
        assert (len(df["X_mm"] == len(df["Y_mm"]))), \
            "X_mm and Y_mm are different lengths! They must be the same."
        
        source = ColumnDataSource(df)
        
        if respective is True:
            high_beetle = np.percentile(df[cmap_col_beetle], high_percentile)
            high_stim = np.percentile(df[cmap_col_stim], high_percentile)
            # also change colorbar labels so max has =< symbol
        
        mapper_beetle = linear_cmap(field_name=cmap_col_beetle, 
                             palette=palette_beetle, 
                             low=low, 
                             high=high_beetle)
        
        mapper_stim = linear_cmap(field_name=cmap_col_stim, 
                             palette=palette_stim, 
                             low=low, 
                             high=high_stim)
        
        p = figure(background_fill_color="#efe8e2", 
                   width=800,
                   height=800,
                   x_axis_label="X (mm)",
                   y_axis_label="Y (mm)")
        
        # stimulus:
        p.circle(source=source,
                 x="X_mm", 
                 y="Y_mm", 
                 color=mapper_stim, 
                 size=size*7, 
                 alpha=alpha/10)
        
        # beetle:
        p.circle(source=source,
                 x="X_mm",
                 y="Y_mm",
                 color=mapper_beetle,
                 size=size,
                 alpha=alpha)
        

        color_bar_beetle = ColorBar(color_mapper=mapper_beetle['transform'], 
                                    title=cmap_label_beetle,
                                    title_text_font_size="7pt",
                                    width=10,
                                    location=(0,0))
        
        color_bar_stim = ColorBar(color_mapper=mapper_stim['transform'], 
                                  title=cmap_label_stim,
                                  title_text_font_size="7pt",
                                  width=10,
                                  location=(0,0))
        
        if show_start is True:
            # Other options include .cross, .circle_x, and .hex:
            p.circle(x=df["X_mm"][0], 
                     y=df["Y_mm"][0], 
                     size=12,
                     color="darkgray",
                     fill_alpha=0.5)

        p.add_layout(color_bar_beetle, "right")
        p.add_layout(color_bar_stim, "right")

        p.title.text_font_size = "14pt"
        p.xaxis.axis_label_text_font_size = '10pt'
        p.yaxis.axis_label_text_font_size = '10pt'

        bokeh_ps.append(p)

        # Output:
        if save_path is not None:
            filename = save_path + f"fictrac_XY"
            
            p.output_backend = "svg"
            export_svgs(p, filename=filename + ".svg")
            export_png(p, filename=filename + ".png")
            output_file(filename=filename + ".html", 
                        title=filename)
            
        if show_plots is True:
            # In case this script is run in Jupyter, change output_backend 
            # back to "canvas" for faster performance:
            p.output_backend = "canvas"
            show(p)
        else:
            bokeh_ps.append(p)
        
    if show_plots is False:
        return bokeh_ps

In [ ]:
plot_fictrac_XY_cmap_with_stim(df_plot, alpha=1)

I use `plotly` to generate 3D plots.

In [ ]:
step_sampler = 1
df = pson_open_by_animal[0][::step_sampler]

fig = go.Figure()
opacity = 1
size = 2.5

# Beetle position relative to map:
fig.add_trace(go.Scatter3d(x=df["X_mm"],
                           y=df["Y_mm"],
                           z=df["dist_from_stim_mm"],
                           name="beetle",
                           opacity=opacity,
                           marker=dict(size=size,
                                       color=df["dist_from_stim_mm"],   # set color to an array/list of desired values
                                       colorscale=bokeh.palettes.Blues256[150:20:-1],   # choose a colorscale
                                       opacity=opacity),
#                           line=dict(color="black")
                          ))

# # Ant position relative to beetle:
# fig.add_trace(go.Scatter3d(x=df["stim_X_mm"], # subsample to get 'dashed' effect
#                            y=df["stim_Y_mm"],
#                            z=df["dist_from_stim_mm"],
#                            name="ant",
#                            opacity=opacity,
#                            marker=dict(size=size,
#                                        color="#496787",
# #                                        color=bokeh.palettes.Set1_3[0],
# #                                        color=df["secs_elapsed"],
# #                                        colorscale=cm["thermal"],
#                                        opacity=opacity),
# #                           line=dict(color="grey")
#                           ))

# Adjust camera view:
camera = dict(
    eye=dict(x=1.3, y=2, z=1.3)
)

fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="#efe8e2",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="#efe8e2",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="#efe8e2",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
    
                    xaxis_title='X (mm)',
                    yaxis_title='Y (mm)',
                    zaxis_title='inter-animal distance (mm)'),
                    
                  
                    width=950,
                    height=800,
                    paper_bgcolor="#f8f5f2",
                    margin=dict(r=20, b=10, l=10, t=10), scene_camera=camera)

fig.show()

# fig.write_image("psom_ol_eg0_trajs3D_view1.pdf")
# poff.plot(fig, filename='psom_ol_eg0_trajs3D.html')

In [ ]:
# Set to larger value to generate "dashed" effect
step_sampler = 50
df = pson_open_by_animal[6]

fig = go.Figure()
opacity = 0.8
size = 3

# Beetle position relative to map:
fig.add_trace(go.Scatter3d(x=df["X_mm"],
                           y=df["Y_mm"],
                           z=np.zeros(len(df)),
                           name="beetle",
                           opacity=opacity/50,
                           marker=dict(size=size,
                                       color="#876949",
                                       opacity=opacity),
#                           line=dict(color="black")
                          ))

# Ant position relative to beetle:
fig.add_trace(go.Scatter3d(x=df["X_mm"][::step_sampler], # subsample to get 'dashed' effect
                           y=df["Y_mm"][::step_sampler],
                           z=df["dist_from_stim_mm"][::step_sampler],
                           name="ant",
                           opacity=opacity/1.5,
                           marker=dict(size=size,
                                       color="#496787",
                                       opacity=opacity),
#                           line=dict(color="grey")
                          ))

fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="#efe8e2",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="#efe8e2",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="#efe8e2",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
    
                    xaxis_title='X (mm)',
                    yaxis_title='Y (mm)',
                    zaxis_title="ant's distance to beetle (mm)"),
                    
                  
                    width=950,
                    height=800,
                    paper_bgcolor="#f8f5f2",
                    margin=dict(r=20, b=10, l=10, t=10))

fig.show()

# poff.plot(fig, filename='ant_beetle_rollercoaster.html')

In [ ]:
df = pson_open_by_animal[6]

opacity = 1
size = 2
# Set to larger value to generate "dashed" effect
step_sampler = 50

# Plot:
fig = go.Figure()

# Ant position relative to map:
fig.add_trace(go.Scatter3d(x=df["stim_X_mm"], 
                           y=df["secs_elapsed"],
                           z=df["stim_Y_mm"],
                           name="Liometopum",
                           opacity=opacity,
                           marker=dict(size=size,
                                       color="#876949",
                                       opacity=opacity),
#                           line=dict(color="grey")
                          ))

# Beetle position relative to map:
fig.add_trace(go.Scatter3d(x=df["X_mm"],
                           y=df["secs_elapsed"],
                           z=df["Y_mm"],
                           name="Platyusa",
                           opacity=opacity,
                           marker=dict(size=size,
                                       color="#496787",
#                                        color=df["secs_elapsed"],   # set color to an array/list of desired values
#                                        colorscale=cm["thermal"],   # choose a colorscale
                                       opacity=opacity),
#                           line=dict(color="black")
                          ))

# Adjust camera view:
camera = dict(
    eye=dict(x=2, y=2, z=0.1)
)

fig.update_layout(scene = dict(
                    aspectmode="manual", aspectratio=dict(x=1,y=2,z=1),
                    xaxis = dict(
                         backgroundcolor="#efe8e2",
                         gridcolor="#f8f5f2",
                         showbackground=True,
                         zerolinecolor="#f8f5f2",),
                    yaxis = dict(
                        backgroundcolor="#efe8e2",
                        gridcolor="#f8f5f2",
                        showbackground=True,
                        zerolinecolor="#f8f5f2"),
                    zaxis = dict(
                        backgroundcolor="#efe8e2",
                        gridcolor="#f8f5f2",
                        showbackground=True,
                        zerolinecolor="#f8f5f2"),
    
                    xaxis_title='X (mm)',
                    zaxis_title='Y (mm)',
                    yaxis_title="time (secs)"),
                    
                    width=950,
                    height=800, 
                    paper_bgcolor="#f8f5f2",
                    margin=dict(r=20, b=10, l=10, t=10), scene_camera=camera)

fig.show()
# fig.write_image("psom_ol_eg6_trajs3D.pdf")
# poff.plot(fig, filename='ant_beetle_rollercoaster.html')